In [1]:
from datetime import datetime
import pandas as pd
from bqplot import ColorScale, ColorAxis, DateScale, LinearScale, Axis, Lines, Figure
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)
from ipywidgets import Label, VBox, HBox, Layout, Accordion, Dropdown, widgets, interact, Button, Output
from bqplot.market_map import MarketMap
import os
from datetime import datetime
from iexfinance.stocks import get_historical_data
import numpy as np
import matplotlib.pyplot as plt
from pandas import Series
import warnings
from IPython.display import clear_output
warnings.simplefilter("ignore", DeprecationWarning)

## RETRIEVE AND CLEAN DATA

In [6]:
etf_list = ['SPY','IVV','VTI','VOO','QQQ','VEA','EFA','IEFA','VWO','AGG','IJH','IEMG','IWM','IJR','VTV','IWF','IWD','VUG','BND','LQD','XLF','VNQ','VIG','EEM','GLD','VB','BSV','VO','TIP','VEU','IVW','DIA','XLK','VYM','VGT','VCSH','MDY','IWB','VCIT','XLV','IWR','XLE','DVY','USMV','EWJ','VGK','PFF','SCHF','SDY','RSP','XLY','ITOT','IVE','SCHX','HYG','SHV','VBR','EMB','SHY','VV','SCHB','XLI','BIV','VT','MBB','BNDX','IWS','VXUS','FLOT','IWO','IXUS','MINT','SCZ','PYZ','MXI','IYM','IXP','RXI','VCR','RHS','VDC','PXI','PXE','IEO','RYF','IYG','KIE','FBT','PTH','IHI','ITA','VIS','ICF','REZ','RWR','PSJ','IGV','RYU','IDU','VPU']
def get_iex_data(stock_list, start=datetime(2015,1,1), end=datetime(2019,12,31)):
    return_list = []
    for i in stock_list:
        df = pd.DataFrame(get_historical_data(i, start, end, output_format='pandas', token='pk_d28c0190de7a4d6da30b3bd2b08487c8')).interpolate()
        df['ticker'] = i
        return_list.append(df)
    return return_list

In [7]:
full = get_iex_data(etf_list)

In [8]:
full

[              open    high     low   close     volume ticker
 date                                                        
 2015-01-08  204.01  206.16  203.99  205.90  147217784    SPY
 2015-01-09  206.40  206.42  203.51  204.25  158567288    SPY
 2015-01-12  204.41  204.60  201.92  202.65  144396067    SPY
 2015-01-13  204.12  205.48  200.51  202.08  214553306    SPY
 2015-01-14  199.65  201.10  198.57  200.86  192991092    SPY
 ...            ...     ...     ...     ...        ...    ...
 2019-12-24  321.47  321.52  320.90  321.23   20270007    SPY
 2019-12-26  321.65  322.95  321.64  322.94   31024188    SPY
 2019-12-27  323.74  323.80  322.28  322.86   42554820    SPY
 2019-12-30  322.95  323.10  320.55  321.08   49782730    SPY
 2019-12-31  320.53  322.13  320.15  321.86   57106998    SPY
 
 [1254 rows x 6 columns],
               open    high     low   close   volume ticker
 date                                                      
 2015-01-08  205.45  207.59  205.43  207.40  4

In [9]:
def lstm_clean_data(data):
    for i in range(len(data)):
        data[i] = data[i].reset_index().dropna()
        data[i]['date'] = pd.to_datetime(data[i]['date'])
        data[i] = data[i].set_index('date')
        data[i]['Reg_Target'] = data[i]['close'].shift(-1)
    return data

def add_past(etf_list, times):
    for i in range(len(etf_list)):
        for n in times:
            etf_list[i]['{}day return'.format(n)] = -etf_list[i]['close'].diff(periods=n).round(3)
    return etf_list

In [10]:
clean_full = lstm_clean_data(full)
data = add_past(clean_full, [1, 5, 21, 252])
# inp = add_past(clean_full, [1,2,3,4,5])

In [12]:
comb = data[0]
x=1
while x != 100:
    comb = comb.append(data[x])
    x += 1
    print (x)
comb.to_csv('dashboard_100_etfs.csv', encoding='utf-8')
comb

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


,open,high,low,close,volume,ticker,Reg_Target,1day return,5day return,21day return,252day return
date,,,,,,,,,,,
2015-01-08,204.01,206.16,203.99,205.90,147217784,SPY,204.25,NaN,NaN,NaN,NaN
2015-01-09,206.40,206.42,203.51,204.25,158567288,SPY,202.65,1.65,NaN,NaN,NaN
2015-01-12,204.41,204.60,201.92,202.65,144396067,SPY,202.08,1.60,NaN,NaN,NaN
2015-01-13,204.12,205.48,200.51,202.08,214553306,SPY,200.86,0.57,NaN,NaN,NaN
2015-01-14,199.65,201.10,198.57,200.86,192991092,SPY,199.02,1.22,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2019-12-24,141.59,141.78,141.04,141.62,52726,VPU,141.86,-0.07,-0.30,-2.20,-26.91
2019-12-26,141.85,142.05,141.37,141.86,68987,VPU,142.25,-0.24,0.22,-2.85,-25.53
2019-12-27,142.01,142.27,141.71,142.25,107721,VPU,142.25,-0.39,-0.29,-2.71,-24.96


## RUN MODELS

In [13]:
from keras.models import Sequential
from keras.layers import Activation, Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
import timeit

Using TensorFlow backend.


In [38]:
def lstm_time_test_split(X, n_past, date):
    X = X.reset_index()
    scaler = MinMaxScaler()
    y_scaler = MinMaxScaler()
    ticker = X['ticker'].iloc[0]
    x_train = X[X['date'] < date].drop(columns=['date', 'Reg_Target', 'ticker', '1day return', '5day return', '21day return', '252day return'])
    scaler.fit(x_train)
    x_test = X[X['date'] >= date].drop(columns=['date', 'Reg_Target', 'ticker', '1day return', '5day return', '21day return', '252day return'])[:-1]
    x_train = scaler.transform(x_train)
    x_train = np.reshape(x_train,(x_train.shape[0], n_past, x_train.shape[1]))
    x_test = scaler.transform(x_test)
    x_test = np.reshape(x_test,(x_test.shape[0], n_past, x_test.shape[1]))
    y_train = np.array(X[X['date'] < date]['Reg_Target'].drop(columns='date')).ravel().astype('float').reshape(-1,1)
    y_scaler.fit(y_train)
    y_train = y_scaler.transform(y_train)
    y_test = np.array(X[X['date'] >= date]['Reg_Target'].drop(columns='date')).ravel().astype('float')[:-1].reshape(-1,1)
    y_test = y_scaler.transform(y_test)
    x_holdout = X[X['date'] >= date].drop(columns=['date', 'Reg_Target', 'ticker', '1day return', '5day return', '21day return', '252day return'])[-1:]
    x_holdout = scaler.transform(x_holdout)
    x_holdout = np.reshape(x_holdout,(x_holdout.shape[0], n_past, x_holdout.shape[1]))
#     y_test = scaler.transform(y_test)
    return ticker, x_train, x_test, x_holdout, y_train, y_test, scaler, y_scaler

def build_step_model(x_train, y_train, epoc):
    model = Sequential()
    model.add(LSTM(50, input_shape=(x_train.shape[1], x_train.shape[2]), return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(100, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.add(Activation('relu'))
    model.compile(loss='mse', optimizer='adam')
    history = model.fit(x_train, y_train, epochs=epoc, batch_size=64, validation_split=.07, verbose=2,shuffle=False)
    return model, history

def yield_preds(model, scaler, x_test, x_holdout, y_test):
    yhat = model.predict(x_test)
    preds = scaler.inverse_transform(yhat)
    true = scaler.inverse_transform(y_test)
    today = model.predict(x_holdout)
    today_pred = scaler.inverse_transform(today)
    return preds, today_pred

def run_all_lstms(data, split, epoc):
    out = pd.DataFrame()
    tomorrow = pd.DataFrame()
    start = timeit.default_timer()
    for i in range(len(data)):
        ticker, x_train, x_test, x_holdout, y_train, y_test, scaler, y_scaler = lstm_time_test_split(data[i], 1, split)
        print('Model #: {}'.format(i))
        model, history = build_step_model(x_train, y_train, epoc)
        preds, future = yield_preds(model, y_scaler, x_test, x_holdout, y_test)
        out[ticker] = preds.flatten()
        tomorrow[ticker] = future.flatten()
    out = out.set_index(data[0][-len(out):].index)
    stop = timeit.default_timer()
    print('Time: ', stop - start)
    return out, tomorrow

## MAKE DASHBOARD INPUTS

In [35]:
etf_data = pd.read_csv('alletfs_details_type_fund_flow.csv')
etf_data.columns = [i.lower() for i in etf_data.columns]

In [36]:
def make_yesterday_mapframe(ref):
    yesterday_inputs = pd.DataFrame([ref[i].reset_index().iloc[-1] for i in range(len(ref))])
    mapframe = pd.merge(etf_data, yesterday_inputs, on='ticker').drop(columns = 'Reg_Target')
    return mapframe

In [37]:
mapframe_y = make_yesterday_mapframe(data)
mapframe_y.columns = ['ticker',
 'etf name',
 'asset class',
 'total assets ',
 'ytd',
 'avg volume',
 'previous closing price',
 '1-day change',
 'inverse',
 'leveraged',
 '1 week',
 '4 week',
 '1 year',
 '3 year',
 '5 year',
 'ytd ff',
 '1 week ff',
 '4 week ff',
 '1 year ff',
 '3 year ff',
 '5 year ff',
 'etfdb.com category',
 'inception',
 'er',
 'commission free',
 'annual dividend rate',
 'dividend date',
 'dividend',
 'annual dividend yield %',
 'p/e ratio',
 'beta',
 '# of holdings',
 '% in top 10',
 'st cap gain rate',
 'lt cap gain rate',
 'tax form',
 'lower bollinger',
 'upper bollinger',
 'support 1',
 'resistance 1',
 'rsi',
 'overall rating',
 'liquidity rating',
 'expenses rating',
 'returns rating',
 'volatility rating',
 'dividend rating',
 'concentration rating',
 'esg score',
 'esg score peer percentile (%)',
 'esg score global percentile (%)',
 'carbon intensity (tons of co2e / $m sales)',
 'sri exclusion criteria (%)',
 'sustainable impact solutions (%)']

ValueError: Length mismatch: Expected axis has 64 elements, new values have 54 elements

In [39]:
def make_daily_mapframe(ref, value):
    mapframe = pd.DataFrame()
    for i in range(len(ref)):
        ticker = ref[i]['ticker'].iloc[0]
        mapframe[ticker] = ref[i][value]
#     mapframe = mapframe.set_index(ref[0]['date']) 
    return mapframe

In [40]:
mapframe_daily_close = make_daily_mapframe(data, 'close')

In [41]:
df = [i for i in full if etf_list in list(i['ticker'])]

In [13]:
mapframe_preds = pd.read_csv('important_backup.csv').set_index('date')

In [42]:
def explore(ticker):
    df = [i for i in full if ticker in list(i['ticker'])][0]
    trace = go.Candlestick(x=df.reset_index()['date'],
                           open=df['open'],
                           high=df['high'],
                           low=df['low'],
                           close=df['close'])
    layout = go.Layout(title='{} Price (USD)'.format(ticker),xaxis = dict(
            rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label='1m',
                     step='month',
                     stepmode='backward'),
                dict(count=6,
                     label='6m',
                     step='month',
                     stepmode='backward'),
                dict(step='all')
            ])),
            rangeslider = dict(
                visible = False)))
    inp = [trace]
    fig = go.Figure(data=inp,layout=layout)
    iplot(fig)
interact(explore, ticker=etf_list)

interactive(children=(Dropdown(description='ticker', options=('SPY', 'IVV', 'VTI', 'VOO', 'QQQ', 'VEA', 'EFA',…

<function __main__.explore(ticker)>

In [43]:
col = ColorScale()
segments = mapframe_y['Segment'].values
issuers = mapframe_y['Issuer'].values
market_map1 = MarketMap(names=etf_list,      
                       # basic data which needs to set for each map
                       ref_data=mapframe_y, groups=segments, cols=50, row_groups=8,
                       tooltip_fields=['Fund Name', 'Segment', 'Issuer', 'Expense Ratio', 'Total Assets', 'Previous Close', '1 Day return', '1 Week return', '1 Month return', '1 Year return'],
                       scales={'color': col},
                       layout={'min_width': '1500px', 'min_height': '1100px'})
market_map1.font_style = {'font-size': '8px', 'fill':'black'}
market_map1.title = 'ETF Market Map'
out = Output()

drop1 = Dropdown(options=['Groups', '1 Day return', '1 Week return', '1 Month return', '1 Year return'], value= 'Groups', description='Color By:', disabled=False,)
drop2 = Dropdown(options=['Segment', 'Issuer'], value='Segment', description='Group By:', disabled=False,)
button1 = Button(description='Clear Plots')
selectors = HBox([drop1, drop2, button1])


def on_d1_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        market_map1.color = mapframe_y[change['new']]
        market_map1.axes = [ColorAxis(scale=col, label=change['new'], visible=True)]
        
def on_d2_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        market_map1.groups = mapframe_y[change['new']]
        
def on_box_change(change):
    with out:
        df = [i for i in full if change['new'][-1] in list(i['ticker'])][-1]
        trace = go.Candlestick(x=df.reset_index()['date'], open=df['open'], high=df['high'], low=df['low'], close=df['close'])
        layout = go.Layout(title='{} Price (USD)'.format(change['new'][-1]),xaxis = dict(
                rangeselector=dict(
                buttons=list([
                    dict(count=1, label='1m', step='month', stepmode='backward'),
                    dict(count=6, label='6m', step='month', stepmode='backward'),
                    dict(step='all')
                ])),
                rangeslider = dict(visible = False)))
        inp = [trace]
        fig = go.Figure(data=inp,layout=layout)
        iplot(fig)
    
def clear_plots(click):
    out.clear_output()
    
drop1.observe(on_d1_change)
drop2.observe(on_d2_change)
button1.on_click(clear_plots)
market_map1.observe(on_box_change, 'selected')
first_page = VBox([selectors, market_map1, out])
first_page

KeyError: 'Segment'